In [1]:
from src.data_preprocessor import DataProcessor
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
from models.knn import KNN
from models.null_model import NullModelClassification, NullModelRegression
from data_configs.configs import *
import statistics

config = car_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
classification_nullmodel = NullModelClassification(config=config)
regression_nullmodel = NullModelRegression(config=config)
knn_model = KNN(config)

In [2]:
# Data Processing

raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

data_2 = data_processor.encode_nominal_features(data_1)

data_3 = data_processor.encode_ordinal_features(data_2)

In [3]:
data_train, data_val = cross_validator.random_partition(data_3, random_state=42)

In [4]:
len(data_train)

1382

In [6]:
knn_model.edited_knn_classificaton(data_train)

,buying,maint,doors,persons,lug_boot,safety,Class
107,0,0,3,2,2,2,unacc
901,2,0,1,1,0,1,unacc
706,1,2,2,0,1,1,unacc
678,1,2,1,0,1,0,unacc
722,1,2,2,2,0,2,acc
...,...,...,...,...,...,...,...
1044,2,1,2,2,0,0,unacc
1095,2,2,0,1,2,0,unacc
860,1,3,3,2,1,2,acc
1459,3,1,2,0,0,1,unacc


In [7]:
result = knn_model.knn_classifier(data_val,data_train,5)
Evaluation().zero_one_loss(data_val[config['target_column']],result['Predicted Class'])

0.04335260115606936

In [8]:
f1_score = Evaluation().f1_score(data_val[config['target_column']],result['Predicted Class'])

print(f1_score)

0.9563120816823143
